In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
PATH = '../data/durecdial/'
FILE_LIST = os.listdir(PATH)

In [3]:
for FILE in FILE_LIST:
    print(FILE)
df_dev = pd.read_csv(PATH + FILE_LIST[0])
test = pd.read_csv(PATH + FILE_LIST[1])
train = pd.read_csv(PATH + FILE_LIST[2])

dev_pp_pro.csv
test_pp_pro.csv
train_pp_pro.csv


In [4]:
train = train.dropna(subset = ['goal_type']).reset_index()

In [5]:
train.columns

Index(['index', 'user_id', 'user_profile_age_range', 'user_profile_name',
       'user_profile_residence', 'user_profile_accepted_food',
       'user_profile_accepted_movies', 'user_profile_accepted_music',
       'user_profile_rejected_music', 'user_profile_gender',
       'user_profile_accepted_celebrities', 'user_profile_accepted_movie',
       'user_profile_reject', 'user_profile_rejected_movies',
       'user_profile_occupation', 'user_profile_accepted_music.1',
       'user_profile_accepted_poi', 'user_profile_favorite_news',
       'user_profile_accepted_news', 'user_profile_poi', 'time', 'place',
       'date', 'topic', 'wday', 'sentence_index', 'sentence', 'goal_topic',
       'goal_type', 'knowledge'],
      dtype='object')

In [6]:
def draw_pie_chart(dataframe, column, min_range=0, max_range=50):
    """
    column : 출력 원하는 column명
    min_range, max_range : 함수에 표시할 sentence_index 범위, 기본값 [0, 50]
    """
    df = dataframe[[column]].loc[(dataframe['sentence_index']>=min_range) & (dataframe['sentence_index']<=max_range)].value_counts().to_frame()
    df.columns = ['count']
    df.reset_index(inplace=True)
    if df.shape[0] > 10:
        df.loc[9] = ['기타',df['count'].iloc[10:].sum()]
        df = df.iloc[:10]
    fig = go.Figure(data=[go.Pie(labels=df[column], values=df['count'])])
    fig.update_traces(textinfo='percent')
    fig.update_layout(title=f'{column} Distribution')
    fig.show()

In [7]:
draw_pie_chart(train, 'goal_type')

In [8]:
draw_pie_chart(train, 'goal_topic')

In [9]:
draw_pie_chart(train, 'topic')

In [10]:
draw_pie_chart(train, 'place')

In [11]:
def draw_bar_chart(dataframe, column, min_range=0, max_range=50, horizontal = False):
    """
    column : 출력 원하는 column명
    min_range, max_range : 함수에 표시할 sentence_index 범위, 기본값 [0, 50]
    horizontal : 가로 출력 여부
    """
    df = dataframe[column].value_counts().to_frame().reset_index()
    df.columns = [column, 'count']
    if horizontal:
        fig = px.bar(df, x='count', y=column, orientation='h')
    else:
        fig = px.bar(df, x=column, y='count')
    fig.show()

In [12]:
draw_bar_chart(train, 'wday')

In [13]:
draw_bar_chart(train, 'time', horizontal=True)

In [34]:
def draw_line_chart(dataframe, column, min_range=0, max_range=50, smooth=True):
    """
    column : 출력 원하는 column명
    min_range, max_range : 함수에 표시할 sentence_index 범위, 기본값 [0, 50]
    smooth : 그래프 부드럽게 그리는 여부
    """
    if smooth:
        shape = 'spline'
    else:
        shape = 'linear'
    df = dataframe[[column, 'sentence_index']].value_counts().to_frame()
    df.columns = ['count']
    df.reset_index(inplace=True)
    df.sort_values(by=[column, 'sentence_index'], inplace=True)
    fig = px.line(df, x="sentence_index", y="count", color="goal_type", line_shape=shape)
    fig.show()

In [35]:
draw_line_chart(train, 'goal_type')

In [4]:
df = pd.read_csv(PATH + 'train_user_small.csv')

In [29]:
def precision_at_K(dataframe, xtitle = None, ytitle = None, smooth=False):
    """
    xtitle, ytitle : x축, y축 제목
    smooth : 그래프 부드럽게 그리는 여부
    """
    precisions = [f'precision_{i}' for i in range(1,10)]
    pk = dataframe.groupby(by='user_id')[precisions].mean().mean()
    if smooth:
        shape = 'spline'
    else:
        shape = 'linear'
    fig = px.line(pk, line_shape=shape)
    fig.update_layout(showlegend = False)
    fig.update_xaxes(title = xtitle)
    fig.update_yaxes(title = ytitle)
    
    return fig

In [30]:
precision_at_K(df).show()